In [9]:
import sqlite3
import pandas as pd


# Подключаемся к базе данных
conn = sqlite3.connect('C:\\Users\\Askar\\VS_CODE\\Python\\HACKATON\\ITMO\\DB\\hack.db')

# Создаем курсор
cursor = conn.cursor()

# Выполняем запрос и загружаем данные в DataFrame
res = pd.read_sql('select * from info', con=conn)

# Закрываем соединение
conn.close()

coffe_data = pd.DataFrame(res)

In [11]:
coffe_data["Product Name"].value_counts()

Product Name
Дрип кофе Tasty Coffee Бэрри, 10 шт. по 11,5 г              141
Кофе в зернах Tasty Coffee Бразилия Серрадо, 250 г           81
Дрип кофе Tasty Coffee Бэрри, 30 шт. по 11,5 г               66
Дрип кофе Tasty Coffee Drip Mix, 20 шт. по 11,5 г            66
Кофе в зернах Tasty Coffee Бразилия Суль-де-Минас, 250 г     65
Кофе в зернах Tasty Coffee Бразилия Можиана, 250 г           63
Кофе в зернах Tasty Coffee Натти, 250 г                      46
Кофе в зернах Tasty Coffee Колумбия Богота, 250 г            44
Кофе в зернах Tasty Coffee Натти, 1000 г                     41
Name: count, dtype: int64

In [2]:
coffe_data.sample(5)

NameError: name 'coffe_data' is not defined

In [1]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.PersistentClient(path="C:\\Users\\Askar\\VS_CODE\\Python\\HACKATON\\ITMO\\DB\\vector_hack")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-large")
col = client.get_or_create_collection(name="hack", embedding_function=sentence_transformer_ef)
# пример запроса
results = col.query(
    query_texts=["Кофе вкусный, negative"],
    n_results=5,
    include=['documents', 'distances', 'metadatas']
)
print(results['metadatas'])
print(results['documents'])

c:\Users\Askar\anaconda3\envs\papa\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


: 

In [25]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ["хороший", "положительный", "популярный"]
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:1] @ embeddings[1:].T) * 100
print(scores.tolist())


[[89.63629913330078, 86.84617614746094]]


In [12]:
import sys

sys.path.append("C:\\Users\\Askar\\VS_CODE\\Python\\HACKATON\\ITMO\\")

In [7]:
from Gigachat import GigaChatBOT
from prompts import PROMPT_FOR_GET_ATRIBUTE, PROMPT_FOR_GET_ATRIBUTE_FINALL

gigachat = GigaChatBOT().init_gigachat()

uniq_names = list(set(list(coffe_data["Product Name"])))

for name in uniq_names[:2]:
    if name:
        reviews = list(coffe_data[coffe_data['Product Name'] == name]["Review Text"])
        res = ""
        for rev in range(0,len(reviews), 20):
            result = GigaChatBOT().query_text(gigachat, reviews[rev:rev+20], PROMPT_FOR_GET_ATRIBUTE)
            res += result + "\n\n"

        fianll_result = result = GigaChatBOT().query_text(gigachat, res, PROMPT_FOR_GET_ATRIBUTE_FINALL)
        print(fianll_result)

            

1. Число пакетиков: 10
2. Единиц в одном товаре: 10
3. Название вкуса: Бэрри
4. Аромат и вкус средний
5. Заваривать удобно и быстро
6. Кофе вкусный, с лёгкой кислинкой, ароматный
7. Удобно заваривать в чашке
1. Вес товара, г: 250
2. Единиц в одном товаре: 1
3. Название вкуса: Натти


In [ ]:
Sentiment == Negative 

coffe_data[coffe_data["Sentiment"] == "Negative"]